In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms, utils

# Define Bayesian Model

In [2]:
#import best fitting distributions and parameters

feature_dists_df = pd.read_csv('feature_dists_df.csv').drop(['Unnamed: 0'], axis=1)
feature_dists_df

,feature,distribution_type,params,label
0,feature_2,beta,"(0.11776839045890578, 149.74182795203535, -2.1...",5
1,feature_4,beta,"(0.2711632488092937, 465.4069516854168, -1.928...",5
2,feature_5,beta,"(0.21602060508177978, 625.918439600516, -4.846...",5
3,feature_6,beta,"(14.256280670273213, 6.9058459231954, 1.454610...",5
4,feature_7,beta,"(0.4894176208554678, 528.8948875881285, -8.432...",5
5,feature_8,beta,"(0.11519992942043017, 197.06722892055205, -4.6...",5
6,feature_9,beta,"(0.3018378051146521, 204.95938188638956, -5.83...",5
7,feature_10,beta,"(14.752993151276618, 6.344337461684533, 0.1954...",5
8,feature_2,beta,"(14.821670613196302, 7.86665989995932, -6.5503...",8
9,feature_4,beta,"(16.236326591296894, 5.212638487058973, -7.674...",8


In [3]:
#create RV objects and store in dicts

feature_dists_5 = {}
feature_dists_8 = {}

for row in feature_dists_df.iterrows():
    row = row[1]
    
    #convert params string into tuple of floats
    params = tuple(map(float, row['params'][1:-1].split(', ')))
    
    #initialize RV objects
    if row['distribution_type']=='beta':
        dist = stats.beta(params[0], params[1], params[2], params[3])
    elif row['distribution_type']=='chi2':
        dist = stats.chi2(params[0], params[1], params[2])
    else:
        print('INVALID DISTRIBUTION TYPE')
        break
    
    #store RV objects in dicts
    if row['label'] == 5:
        feature_dists_5[row['feature']] = dist
    else:
        feature_dists_8[row['feature']] = dist


In [4]:
feature_dists_5

{'feature_2': <scipy.stats._distn_infrastructure.rv_frozen at 0x7fb4f0dfe970>,
 'feature_4': <scipy.stats._distn_infrastructure.rv_frozen at 0x7fb4f0dfe880>,
 'feature_5': <scipy.stats._distn_infrastructure.rv_frozen at 0x7fb4f0dfec10>,
 'feature_6': <scipy.stats._distn_infrastructure.rv_frozen at 0x7fb4f0dfef10>,
 'feature_7': <scipy.stats._distn_infrastructure.rv_frozen at 0x7fb4f0dfe940>,
 'feature_8': <scipy.stats._distn_infrastructure.rv_frozen at 0x7fb4f0dfe820>,
 'feature_9': <scipy.stats._distn_infrastructure.rv_frozen at 0x7fb4f0ded940>,
 'feature_10': <scipy.stats._distn_infrastructure.rv_frozen at 0x7fb4f0dedc70>}

In [5]:
#define bayesian model
#inputs: feature values
#outputs: (P(I|5)*P(5), P(I|8)*P(8))

def bayesian_model(feature_vals):
    
    #define priors
    prior_5 = np.log(0.5)
    prior_8 = np.log(0.5)
    
    #compute P(I|C) likelihoods
    likelihood_5 = 0
    likelihood_8 = 0
    
    for i, val in enumerate(feature_vals):
        
        feature_num = i+1
        
        if feature_num==1 or feature_num==3:
            continue
            
        #get distribution (RV) objects
        dist_5 = feature_dists_5['feature_'+str(feature_num)]
        dist_8 = feature_dists_8['feature_'+str(feature_num)]
        
        #get log probability of value being between (int(val), int(val)+1)
        log_prob_5 = np.log(dist_5.cdf(int(val)+1) - dist_5.cdf(int(val)))
        log_prob_8 = np.log(dist_8.cdf(int(val)+1) - dist_8.cdf(int(val)))
        
        #update likelihoods
        likelihood_5 += log_prob_5
        likelihood_8 += log_prob_8
    
    output_5 = np.exp(prior_5 + likelihood_5)
    output_8 = np.exp(prior_8 + likelihood_8)
    
    return output_5, output_8
    

# Test Bayesian Model

In [6]:
#import test dataset
test_dataset = datasets.MNIST(root='data',
                              train=False,
                              download=True,
                              transform=transforms.Compose([
                              transforms.ToTensor(),
                              transforms.Normalize((0.1307,), (0.3081,))]))

#filter dataset to only 5's and 8's
test_filter = [idx for idx, sample in enumerate(test_dataset) if sample[1] in [5,8]]
test_dataset = torch.utils.data.Subset(test_dataset, test_filter)

In [7]:
#import cnn
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5) # 10 channels in first convolution layer
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5) # 20 channels in second conv. layer
        self.fc1 = nn.Linear(320, 10) # 10 hidden units in first fully-connected layer
        self.fc2 = nn.Linear(10, 2) # 2 output units

    def forward(self, x):

        # first convolutional layer
        h_conv1 = self.conv1(x)
        h_conv1 = F.relu(h_conv1)
        h_conv1_pool = F.max_pool2d(h_conv1, 2)

        # second convolutional layer
        h_conv2 = self.conv2(h_conv1_pool)
        h_conv2 = F.relu(h_conv2)
        h_conv2_pool = F.max_pool2d(h_conv2, 2)

        # fully-connected layer
        h_fc1 = h_conv2_pool.view(-1, 320)
        h_fc1 = self.fc1(h_fc1)
        h_fc1 = F.relu(h_fc1)
        
        # classifier output
        output = self.fc2(h_fc1)
        output = F.log_softmax(output,dim=1)
        return output, h_fc1

cnn = CNN()
cnn.load_state_dict(torch.load('cnn'))
cnn.train(False)

CNN(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=320, out_features=10, bias=True)
  (fc2): Linear(in_features=10, out_features=2, bias=True)
)

In [8]:
from tqdm import tqdm

num_correct = 0

for sample in tqdm(test_dataset):
    data = sample[0]
    label = sample[1]
    
    output, features = cnn(data)
    
    bayesian_out_5, bayesian_out_8 = bayesian_model(features.squeeze().detach().numpy())
    
    if bayesian_out_5 > bayesian_out_8:
        bayesian_pred = 5
    else:
        bayesian_pred = 8

    if bayesian_pred == label:
        num_correct += 1
        
test_acc = num_correct / len(test_dataset)

print('bayesian model test accuracy: {}'.format(test_acc))

  0%|          | 0/1866 [00:00<?, ?it/s]<ipython-input-5-8d0ab4c8502e>:28: RuntimeWarning: divide by zero encountered in log
  log_prob_5 = np.log(dist_5.cdf(int(val)+1) - dist_5.cdf(int(val)))
100%|██████████| 1866/1866 [00:12<00:00, 152.71it/s]

bayesian model test accuracy: 0.9694533762057878
